## Subplaces back to NYC boroughs

In [1]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [2]:
# read in packages
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import numpy as np
import os
import plotly.express as px
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [3]:
subplaces = r'data/shapefiles/nyc_subplace_centroid.geojson'
subplaces = gpd.read_file(subplaces)
subplaces.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,900946940.0,-234.0,POINT (-73.12283 41.52707)
1,2,09,009,47535,00213462,0900947535,Milford,Milford town,43,T5,G4040,720,71950,None,C,57442071.0,10218658.0,+41.2250861,-073.0611101,None,None,None,None,0.481270,0.007266,67660729.0,26.12381,900947535.0,-1743.0,POINT (-73.06185 41.22575)
2,3,09,009,58300,00213486,0900958300,Oxford,Oxford town,43,T1,G4040,720,71950,None,A,84803121.0,1531057.0,+41.4440006,-073.1479992,None,None,None,None,0.439399,0.009300,86334178.0,33.33363,900958300.0,248.0,POINT (-73.13503 41.43129)
3,4,09,009,00000,00000000,0900900000,County subdivisions not defined,County subdivisions not defined,00,Z9,G4040,None,None,None,F,0.0,599104136.0,+41.1874659,-072.8153339,None,None,None,None,1.700000,0.064300,599104136.0,231.31411,900900000.0,NaN,POINT (-72.79470 41.25792)
4,5,09,009,44560,00213454,0900944560,Madison,Madison town,43,T1,G4040,720,75700,None,A,93622105.0,1251401.0,+41.3444810,-072.6245213,None,None,None,None,0.739953,0.010206,94873506.0,36.63066,900944560.0,-218.0,POINT (-72.62809 41.34013)


In [4]:
df = pd.read_csv(r'data/subpl_x_borough/SG_origsubxnyc_01-10-2020.csv')
decimals = 0

df = df.rename(columns = {'orig_subpl_str':'id'})
df['id'] = df['id'].str[1:]
list_date = df.columns.to_list()
list_date = list_date[2:]
# melt so we have boroughs as a column.
df = pd.melt(df, id_vars=['id','date_y-m-d'], value_vars = list_date)
df = df.rename(columns = {'variable':'Borough','date_y-m-d':'date'})
# add explicit borough column -- New York City, Bronx, Brooklyn, Manhattan, Queens, Staten Island
# borough_name = {'New York City':'NYC', 'Bronx':'BX', 'Brooklyn':'BK', 'Manhattan':'MN', 'Queens':'QN', 'Staten Island':'SI'}
df['Borough'] = df['Borough'].replace({'NYC':'New York City', 'BX':'Bronx', 'BK':'Brooklyn', 'MN':'Manhattan', 'QN':'Queens', 'SI':'Staten Island'})
# parse month out of dates
new = df["date"].str.split("/", n = 1, expand = True) 
df["month"]= new[0].astype(int) # convert to integer
# remove october dates --- for now
df = df.drop(df[df.month > 9].index)
df['date'] = pd.to_datetime(df['date'])
# merge
df = pd.merge(subplaces, df, left_on='GEOID', right_on='id', how = "inner")
df['date'] = df["date"].dt.strftime('%m/%d/%Y')
df['value'] = df['value'].astype(float)
df['value'] = df['value'].apply(lambda x: round(x, decimals)) # round to nearest whole num
df['value'] = df['value'].astype(int) # remove decimal

#remove rows less that are 0.
df = df.loc[df['value']>=1]
df = df.rename(columns = {'NAME':'Name','date':'Date','value':'Returns/Device'})
df = df.sort_values(by = 'Date')
df = df[df['Borough'] != 'New York City']
df.to_csv(f'data/subpl_x_borough/returns_from_subplace.csv', index = False)
    
# returns(df)
df.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,Name,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry,id,Date,Borough,Returns/Device,month
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,9.009469e+08,-234.0,POINT (-73.12283 41.52707),0900946940,01/01/2020,Bronx,92,1
165018,130,34,003,54990,00885337,3400354990,Oradell,Oradell borough,21,C5,G4040,None,None,None,F,6245216.0,403561.0,+40.9566509,-074.0328576,None,None,None,None,0.178571,0.000711,6648777.0,2.56709,3.400355e+09,-230.0,POINT (-74.03145 40.95621),3400354990,01/01/2020,Bronx,98,1
861315,610,36,None,None,None,3671267,Stewart Manor,Stewart Manor village,47,C1,G4110,None,None,None,A,543101.0,0.0,+40.7201719,-073.6853003,71267,02391159,N,N,0.033494,0.000058,543101.0,0.20969,3.671267e+06,12.0,POINT (-73.68530 40.72017),3671267,01/01/2020,Brooklyn,213,1
165288,130,34,003,54990,00885337,3400354990,Oradell,Oradell borough,21,C5,G4040,None,None,None,F,6245216.0,403561.0,+40.9566509,-074.0328576,None,None,None,None,0.178571,0.000711,6648777.0,2.56709,3.400355e+09,-230.0,POINT (-74.03145 40.95621),3400354990,01/01/2020,Brooklyn,78,1
1277745,890,36,None,None,None,3680170,Westhampton,Westhampton CDP,57,U1,G4210,None,None,None,S,32789529.0,5691266.0,+40.8467926,-072.6598908,80170,02390494,N,N,0.387175,0.004108,38480795.0,14.85743,3.680170e+06,-200.0,POINT (-72.65979 40.83868),3680170,01/01/2020,Queens,267,1


In [5]:
df.tail()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,Name,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry,id,Date,Borough,Returns/Device,month
1175339,821,36,None,None,None,3651517,North Bellmore,North Bellmore CDP,57,U1,G4210,None,None,None,S,6786405.0,0.0,+40.6903458,-073.5390162,51517,02389564,N,N,0.117441,0.000723,6786405.0,2.62023,3.651517e+06,44.0,POINT (-73.53902 40.69035),3651517,09/30/2020,Manhattan,522,9
48135,47,09,005,40290,00213446,0900540290,Kent,Kent town,43,T1,G4040,None,None,None,A,125744050.0,2745773.0,+41.7297820,-073.4541400,None,None,None,None,0.511797,0.013905,128489823.0,49.60992,9.005403e+08,-28.0,POINT (-73.45398 41.72991),0900540290,09/30/2020,Brooklyn,85,9
719582,515,36,027,78388,00979601,3602778388,Washington,Washington town,43,T1,G4040,None,None,None,A,150659403.0,1849182.0,+41.7914813,-073.6771663,None,None,None,None,0.525535,0.016519,152508585.0,58.88356,3.602778e+09,-73.0,POINT (-73.67397 41.79154),3602778388,09/30/2020,Manhattan,856,9
719325,515,36,027,78388,00979601,3602778388,Washington,Washington town,43,T1,G4040,None,None,None,A,150659403.0,1849182.0,+41.7914813,-073.6771663,None,None,None,None,0.525535,0.016519,152508585.0,58.88356,3.602778e+09,-73.0,POINT (-73.67397 41.79154),3602778388,09/30/2020,Brooklyn,107,9
719068,515,36,027,78388,00979601,3602778388,Washington,Washington town,43,T1,G4040,None,None,None,A,150659403.0,1849182.0,+41.7914813,-073.6771663,None,None,None,None,0.525535,0.016519,152508585.0,58.88356,3.602778e+09,-73.0,POINT (-73.67397 41.79154),3602778388,09/30/2020,Bronx,160,9


In [6]:
df = pd.read_csv(r'data\subpl_x_borough\returns_from_subplace.csv')
# df = df.sort_values(by = 'Date')
# df = df[df['Borough'] != 'NYC']

In [7]:
df.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,Name,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry,id,Date,Borough,Returns/Device,month
0,1,9,9.0,46940.0,213459.0,900946940,Middlebury,Middlebury town,43,T1,G4040,720.0,78700.0,NaN,A,45986818.0,1786816.0,41.524691,-73.123016,NaN,NaN,NaN,NaN,0.321040,0.005154,47773634.0,18.44540,9.009469e+08,-234.0,POINT (-73.12282609979773 41.52707296499348),900946940,01/01/2020,Bronx,92,1
1,130,34,3.0,54990.0,885337.0,3400354990,Oradell,Oradell borough,21,C5,G4040,NaN,NaN,NaN,F,6245216.0,403561.0,40.956651,-74.032858,NaN,NaN,NaN,NaN,0.178571,0.000711,6648777.0,2.56709,3.400355e+09,-230.0,POINT (-74.03145102372818 40.95620902708917),3400354990,01/01/2020,Bronx,98,1
2,610,36,NaN,NaN,NaN,3671267,Stewart Manor,Stewart Manor village,47,C1,G4110,NaN,NaN,NaN,A,543101.0,0.0,40.720172,-73.685300,71267.0,2391159.0,N,N,0.033494,0.000058,543101.0,0.20969,3.671267e+06,12.0,POINT (-73.68530031522866 40.72017197987051),3671267,01/01/2020,Brooklyn,213,1
3,130,34,3.0,54990.0,885337.0,3400354990,Oradell,Oradell borough,21,C5,G4040,NaN,NaN,NaN,F,6245216.0,403561.0,40.956651,-74.032858,NaN,NaN,NaN,NaN,0.178571,0.000711,6648777.0,2.56709,3.400355e+09,-230.0,POINT (-74.03145102372818 40.95620902708917),3400354990,01/01/2020,Brooklyn,78,1
4,890,36,NaN,NaN,NaN,3680170,Westhampton,Westhampton CDP,57,U1,G4210,NaN,NaN,NaN,S,32789529.0,5691266.0,40.846793,-72.659891,80170.0,2390494.0,N,N,0.387175,0.004108,38480795.0,14.85743,3.680170e+06,-200.0,POINT (-72.65978521019136 40.83867786809526),3680170,01/01/2020,Queens,267,1


In [9]:
fig = px.scatter_mapbox(df,# geojson = subplaces,
                     lat = 'INTPTLAT', lon = 'INTPTLON',
                     size = 'Returns/Device',
                     color ='Borough',
                     color_discrete_sequence = ['#CCCCFF','#1D819A','#FFE066','#63BB69','#DE7A3F'],
                     animation_frame = 'Date',
                     animation_group = 'Borough',
                     hover_data = {'Name':True,'Date':True,'Returns/Device':True, 'INTPTLAT':False, 'INTPTLON':False, 'Borough':False},
                #     labels={'Borough':'Borough of Residence'},
                     #font = dict(family = 'Arial', color = 'white'),
                     size_max = 60)
           #          labels = 'Borough')
fig.update_layout(mapbox_zoom = 6.8,
                 mapbox_center = {'lat':40.82, 'lon':-74.25},
                 mapbox_style="carto-darkmatter",
                 width = 950,
                 height = 780, 
                 font = dict(family = 'Arial', color = 'black'),
                 title_font_color = 'white',
                 legend=dict(
#                       orientation="h",
                      title = 'Destination',
                      title_font_color = 'white',
                      bgcolor = 'black',
                      bordercolor = 'gray', borderwidth = 1,
                      font = dict(family = 'Arial', color = 'white'),
                      yanchor="top",
                      y=0.33,
                      xanchor="left",
                      x=0.02))
# fig.add_annotation(text = "Non-NYC Residents' Trips to NYC<br>by Home Municipality",
#                   align = 'left', x = 0.01, y = 0.95, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 19))
# fig.add_annotation(text = 'January 1, 2020 to October 31, 2020',
#                   align = 'left', x = 0.01, y = 0.86, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 13))
fig.add_annotation(text = 'Bubble size = Trips per 10,000 mobile devices',
                  align = 'right', x = 0.02, y = 0.01, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
fig.add_annotation(text = 'Click on/off boroughs in legend to compare or isolate migratory patterns',
                  align = 'right', x = 0.02, y = 0.082, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
# fig.show()
fig.write_html('subplace_returns.html',auto_open = True,auto_play = False, include_plotlyjs = 'cdn')